In [10]:
import re
import json
import time
import requests
import pandas as pd
from pyquery import PyQuery as pq
headers = {
    "user-agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.81 Safari/537.36"
}

data = {
    '评论': [],
    '交易时间': [],
    '评论时间': [],
    '交易评论时间差(分钟)': [],
    '会员': [],
    '名字': []
}


#时间戳转换为指定格式的日期
def getime(timeStamp):
    timeArray = time.localtime(timeStamp)
    otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
    return otherStyleTime


#通过url转换得到评论JSON文件的Request URL
def get_base_url(url):
    _html = requests.get(url, headers=headers)
    _doc = pq(_html.text)
    _item = _doc('#LineZing')
    _shopid = int(_item.attr("shopid"))
    _itemid = int(_item.attr("itemid"))
    base_url = f"https://rate.tmall.com/list_detail_rate.htm?itemId={_itemid}&sellerId={_shopid}&order=3&currentPage="
    return base_url


#使用评论JSON文件的Request URL提取JSON文件
def ratejson_get(url):
    html = requests.get(url, headers=headers)
    m = re.search("jsonp1025.*\(({.*)\)", html.text)
    ratejon = json.loads(m.group(1))
    return ratejon


#解析JSON文件
def rateJson_parse(ratejon):
    #评论区
    temp_rate = ratejon["rateDetail"]["rateList"]
    for i in range(len(temp_rate)):
        data['评论'].append(temp_rate[i]["rateContent"])
        data['名字'].append(temp_rate[i]["displayUserNick"])

        if temp_rate[i]["goldUser"] == True:
            data['会员'].append("Yes")
        else:
            data['会员'].append("No")

        tradeEndTime = temp_rate[i]["tradeEndTime"]["time"]  # 获取评论时间
        createTime = temp_rate[i]["gmtCreateTime"]["time"]  # 获取交易时间
        if tradeEndTime != None or createTime != None:
            tradeEndTime = int(tradeEndTime) // 1000
            data['交易时间'].append(getime(tradeEndTime))
            createTime = int(createTime) // 1000
            data['评论时间'].append(getime(createTime))
            dt = (createTime - tradeEndTime) / 60
            data['交易评论时间差(分钟)'].append(dt)
        else:
            data['交易时间'].append("NaN")
            data['评论时间'].append("NaN")
            data['交易评论时间差(分钟)'].append("NaN")


#主函数
def main(url):
    base_url = get_base_url(url)
    ratejon = ratejson_get(base_url + "1")
    rateNum = ratejon["rateDetail"]["rateCount"]["total"]
    picNum = ratejon["rateDetail"]["rateCount"]["picNum"]
    usedNum = ratejon["rateDetail"]["rateCount"]["used"]
    print("总共有%d条评论" % rateNum)
    print("其中带图片评论的有%d条"%picNum)
    print("其中带追加评论的有%d条" % usedNum)

    try:
        for i in range(1, 99):
            ratejon = ratejson_get(base_url + f"{i}")
            rateJson_parse(ratejon)
    except:
        print("End")

- 在此输入网址

In [17]:
url = r"https://rate.tmall.com/list_detail_rate.htm?itemId=569261104313&spuId=0&sellerId=1879194783&order=3&currentPage=1&append=0&content=1&tagId=&posi=&picture=&groupId=&ua=098%23E1hvmQvnvPOvUvCkvvvvvjiPRssZAjnvRLMWgj3mPmPyAjYPn2Lv0jYWRFSyzjYURphvCvvvvvmCvpvZz%2FN5cR34zYMNZYsG7zSN3r%2Fdzb0rvpvEvvkBvi0avbc1dphvmpvWPUE3OpvU6gwCvvpvCvvvRphvCvvvvvvPvpvhvv2MMQhCvvOvCvvvphvEvpCW2HMJvvw%2FgWswsEp7bpPClw03HE3iBf7gQa7tnV34ibmxfBAKNx2h%2BWe9eBXAcixrgj7ZHkx%2Fgjc6rBwgbl%2F9ibmxdByaUUxrD76wddyCvm9vvvvvphvvvvvv9cCvpv1mvvv24ZCv2H%2BvvvniphvW7Qvvv0Dvpv1fuphvmvvv9bfuU5RakphvC99vvOCzp8wCvvpvvUmm&needFold=0&_ksTS=1570528382192_1024&callback=jsonp1025"


- 开始分析网页,并获取数据

In [8]:
main(url)
df = pd.DataFrame(data, columns=['交易时间', '评论时间', '交易评论时间差(分钟)', '会员', '名字'])
pd.DataFrame(data, columns=['交易时间', '评论时间', '交易评论时间差(分钟)', '会员', '名字'])

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [60]:
df.to_csv('Tmall.csv',encoding="utf-8-sig")

In [18]:
a=requests.get(url).text

In [19]:
a

'\r\n<!DOCTYPE html>\n<html>\n<head>\n\t<link rel="stylesheet" href="//g.alicdn.com/??kg/global-util/1.0.6/index-min.css,kg/tb-nav/2.4.1/index-min.css">\n\t<script src="//g.alicdn.com/??kissy/k/1.4.16/seed-min.js,kg/kmd-adapter/0.1.5/index.js,kg/kmd-adapter/0.1.5/util.js,kg/global-util/1.0.7/index-min.js,tb/tracker/4.3.18/index.js,kg/tb-nav/2.5.4/index-min.js"></script>\n<script>KISSY.config({modules:{\'flash\':{alias:[\'gallery/flash/1.0/\']}}});KISSY.use(\'kg/global-util/1.0.7/\');KISSY.config({modules: {\'kg/tb-nav\':{alias:\'kg/tb-nav/2.5.4/\',requires:[\'kg/global-util/1.0.7/\']}}});KISSY.ready(function(){KISSY.use(\'kg/tb-nav\')});</script>\n\t\n \n\n\t<title>淘宝网 - 淘！我喜欢</title>\n<meta charset="gbk" />\n<meta http-equiv="Cache-Control" content="no-cache" />\n<meta http-equiv="pragma" content="no-cache" />\n<meta http-equiv="expires" content="Thu, 01 Jan 1970 00:00:01 GMT" />\n<meta name="spm-id" content="a1z0b" /><link rel="shortcut icon" type="image/x-icon" href="//img.alicdn.co

In [20]:
import math


In [38]:
+12*math.log2(174/100)

9.589047672888043